In [ ]:
!pip install transformers
!pip install accelerate
# TODO: check lr scheduler

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score as kappa
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import GPT2Tokenizer, GPT2Model
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import os
import time
import pathlib

# log folder to save log files
log_folder = '/content/drive/MyDrive/asap/'

# target column
target_column = "score"

# hyper parameters
hp = {
    "base_model": "gpt2",
    "lr": 1e-4,
    "num_epochs": 30,
    "batch_size":2,
    "use_amp": True,
    "mixed_precision": "fp16",
}

# Prepare ASAP Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Original kaggle training set
kaggle_dataset  = pd.read_csv('/content/drive/MyDrive/GPT-2 MODEL/training_set_rel3.tsv', sep='\t', encoding = "ISO-8859-1")

# Smaller training set used for this project
dataset_df = pd.DataFrame(
  {
    'essay_id' : kaggle_dataset['essay_id'],
    'essay_set' : kaggle_dataset['essay_set'],
    'essay' : kaggle_dataset['essay'],
    'rater1' : kaggle_dataset['rater1_domain1'],
    'rater2' : kaggle_dataset['rater2_domain1'],
    'rater3' : kaggle_dataset['rater3_domain1'],
    'score' : kaggle_dataset['domain1_score']
  })
dataset_df.head()

,essay_id,essay_set,essay,rater1,rater2,rater3,score
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8


## Use essay_set=8 for classification

In [ ]:
essay_df = dataset_df[dataset_df['essay_set'] == 8].copy()
essay_df.shape

# essay_df = dataset_df.loc[(dataset_df['essay_set'] == 3) | (dataset_df['essay_set'] == 4) | (dataset_df['essay_set'] == 5) | (dataset_df['essay_set'] == 6)].copy()
# essay_df.shape

(723, 7)

In [ ]:
print(essay_df)

       essay_id  essay_set                                              essay  \
12253     20716          8   A long time ago when I was in third grade I h...   
12254     20717          8   Softball has to be one of the single most gre...   
12255     20718          8   Some people like making people laugh, I love ...   
12256     20719          8   "LAUGHTER"  @CAPS1 I hang out with my friends...   
12257     20721          8  Well ima tell a story about the time i got @CA...   
...         ...        ...                                                ...   
12971     21626          8   In most stories mothers and daughters are eit...   
12972     21628          8   I never understood the meaning laughter is th...   
12973     21629          8  When you laugh, is @CAPS5 out of habit, or is ...   
12974     21630          8                                 Trippin' on fen...   
12975     21633          8   Many people believe that laughter can improve...   

       rater1  rater2  rate

In [ ]:
# Count the value of each group
essay_df['score'].value_counts().sort_index()


10      1
15      1
20      4
21      2
22      1
24      3
25      5
26      4
27      6
28     11
29      8
30     49
31     34
32     37
33     32
34     39
35     47
36     65
37     39
38     20
39      8
40    161
41     22
42     23
43     15
44     14
45     31
46     13
47      7
48      3
49      2
50     13
55      2
60      1
Name: score, dtype: int64

In [ ]:
#Convert the 'score' column to integers only
essay_df['score'] = essay_df['score'].astype(int)


In [ ]:
# Map the classes to new values in order
unique_scores = sorted(essay_df['score'].unique())
mapping = {score: i+1 for i, score in enumerate(unique_scores)}

#Update the dataset.
essay_df['score'] = essay_df['score'].map(mapping)

#Check the recount
print(essay_df['score'].value_counts().sort_index())


1       1
2       1
3       4
4       2
5       1
6       3
7       5
8       4
9       6
10     11
11      8
12     49
13     34
14     37
15     32
16     39
17     47
18     65
19     39
20     20
21      8
22    161
23     22
24     23
25     15
26     14
27     31
28     13
29      7
30      3
31      2
32     13
33      2
34      1
Name: score, dtype: int64


In [ ]:
essay_df['score'].value_counts()

22    161
18     65
12     49
17     47
16     39
19     39
14     37
13     34
15     32
27     31
24     23
23     22
20     20
25     15
26     14
28     13
32     13
10     11
21      8
11      8
29      7
9       6
7       5
8       4
3       4
6       3
30      3
31      2
33      2
4       2
34      1
2       1
5       1
1       1
Name: score, dtype: int64

In [ ]:
!pip install googletrans==4.0.0-rc1


In [ ]:
pip install google_trans_new


In [ ]:
from googletrans import Translator
import pandas as pd

def back_translate(text):
    translator = Translator()
    target_language = 'tr'

    try:
        translated_text = translator.translate(text, dest=target_language).text
        back_translated_text = translator.translate(translated_text, dest='en').text
        return back_translated_text
    except Exception as e:
        print(f"Çeviri hatası: {e}")
        return text  # Hata durumunda orijinal metni kullanın


def augment_data(df, column, value, fraction):
    subset = df[df[column] == value]
    augmented_subset = subset.sample(frac=fraction, replace=True)

    new_rows = []
    max_essay_id = df['essay_id'].max()

    for _, row in augmented_subset.iterrows():
        max_essay_id += 1
        new_rows.append({
            'essay_id': max_essay_id,
            'essay_set': row['essay_set'],
            'essay': back_translate(row['essay']),
            'rater1': row['rater1'],
            'rater2': row['rater2'],
            'score': value
        })

    return pd.DataFrame(new_rows)

# essay_df üzerine veri artırma uygulama
classes_to_augment = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
augmented_dataframes = []

for value in classes_to_augment:
    augmented_dataframes.append(augment_data(essay_df, 'score', value, 1))

augmented_df = pd.concat(augmented_dataframes, ignore_index=True)

#Artırılmış veriyi orijinal veriye ekleyin
essay_df = pd.concat([essay_df, augmented_df], ignore_index=True)

print(f"Total data size after augmentation: {len(essay_df)}")


Çeviri hatası: the JSON object must be str, bytes or bytearray, not NoneType
Total data size after augmentation: 1220


In [ ]:
essay_df['score'].value_counts()

22    161
12     98
17     94
16     78
19     78
14     74
13     68
18     65
15     64
27     62
24     46
23     44
20     40
25     30
26     28
28     26
32     26
10     22
21     16
11     16
29     14
9      12
7      10
8       8
3       8
6       6
30      6
31      4
33      4
4       4
34      2
2       2
5       2
1       2
Name: score, dtype: int64

In [ ]:
# 6'dan düşük skorları 6'ya yükseltme
essay_df['score2'] = essay_df['score'].apply(lambda x: 29 if x < 29 else x)
# Yeni frekansları hesaplama
new_frequencies = essay_df['score2'].value_counts().sort_index()

# 6'dan düşük skorları 6'ya yükseltme
essay_df['score2'] = essay_df['score2'].apply(lambda x: 38 if x == 39 else x)
# Yeni frekansları hesaplama
# Assuming 'score1' is the column with the scores
# Replace score 55 with 50, and any score less than 6 with 60

essay_df['score2'] = essay_df['score2'].apply(lambda x: 50 if x == 60 else x)

import pandas as pd

# Assuming 'essay_df' is your DataFrame and 'score1' is the column of interest

# Filter the DataFrame to exclude rows where 'score1' is 47, 48, or 60
filtered_df = essay_df[~essay_df['score2'].isin([47, 48, 49, 55, 60])]

# If you want to update 'essay_df' directly and use it further, just overwrite it:
essay_df = filtered_df.copy()

# Calculate the new frequencies after the removal
new_frequencies = essay_df['score2'].value_counts().sort_index()

# Print the new frequencies
print(new_frequencies)
essay_df['score'] = essay_df['score2']

In [ ]:
essay_df['score'].value_counts().sort_index()


1     24
2    166
3    443
4    385
5    166
6     36
Name: score, dtype: int64

In [ ]:
print(essay_df)

      essay_id  essay_set                                              essay  \
0        20716          8   A long time ago when I was in third grade I h...   
1        20717          8   Softball has to be one of the single most gre...   
2        20718          8   Some people like making people laugh, I love ...   
3        20719          8   "LAUGHTER"  @CAPS1 I hang out with my friends...   
4        20721          8  Well ima tell a story about the time i got @CA...   
...        ...        ...                                                ...   
1215     21645          8  " @Caps1 day diamonds and @caps1 day stone," @...   
1216     21646          8  The laughter creates a bond between the two pe...   
1217     21634          8  @Caps1-inspirational, @caps2 @caps4 i @caps5 @...   
1218     21635          8  @Caps1-inspirational, @caps2 @caps4 i @caps5 @...   
1219     21634          8  Bell rings.! Chocolate Hair and Mocha bound, i...   

      rater1  rater2  rater3  score  
0

In [ ]:
import numpy as np
# Use minimum score
rater_1 = essay_df["rater1"].to_numpy()
rater_2 = essay_df["rater2"].to_numpy()

min_score = np.minimum(rater_1, rater_2)
max_score = np.maximum(rater_1, rater_2)

essay_df['min_score'] = min_score
essay_df['max_score'] = max_score

In [ ]:
essay_df['score'].value_counts()

3    443
4    385
5    166
2    166
6     36
1     24
Name: score, dtype: int64

In [ ]:
essay_df[target_column] - essay_df[target_column].min()

0       2
1       4
2       3
3       1
4       1
       ..
1215    5
1216    5
1217    5
1218    5
1219    5
Name: score, Length: 1220, dtype: int64

In [ ]:
print(essay_df[target_column])

0       3
1       5
2       4
3       2
4       2
       ..
1215    6
1216    6
1217    6
1218    6
1219    6
Name: score, Length: 1220, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split

essay_df['target_score'] = essay_df[target_column] - essay_df[target_column].min()

X, y = essay_df['essay'].to_list(), essay_df['target_score'].to_numpy()
num_labels = essay_df[target_column].unique().size

# 60 / 40 train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=42) # stratify=y, this paramter will not work if any class has number of examples lower than 2

# split test to half to get 60 / 20 / 20 split

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.50, random_state=42) # stratify=y_test

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# a torch dataset implementation for ASAP dataset
class EssayDataset(Dataset):
    def __init__(self, essays, targets, tokenizer):
        self.essays = essays
        self.targets = targets
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.essays)

    def __getitem__(self, idx):
        text = str(self.essays[idx])
        encoded_input = tokenizer(text, truncation=True, return_tensors='pt').to(device)

        return encoded_input['input_ids'].squeeze(), encoded_input['attention_mask'].squeeze(), self.targets[idx]

In [ ]:
# collater function to pad tokens
def collate_fn(batch):
    PAD_TOKEN_ID = 50256 # Use tokenizer.pad_token_id to check
    input_ids_list, attention_mask_list, targets = [], [], []

    for input_ids, attention_mask, target in batch:
        input_ids_list.append(input_ids)
        attention_mask_list.append(attention_mask)
        targets.append(target)

    # Pad the batch to the maximum sequence length within that batch using the tokenizer's pad token
    max_length = max(len(ids) for ids in input_ids_list)
    padded_input_ids = []
    padded_attention_mask = []

    for input_ids, attention_mask in zip(input_ids_list, attention_mask_list):
        pad_length = max_length - len(input_ids)
        padded_input_ids.append(torch.cat([input_ids, torch.tensor([PAD_TOKEN_ID] * pad_length, device=device, dtype=torch.long)]))
        # add zeros to attention mask for padds
        padded_attention_mask.append(torch.cat([attention_mask, torch.zeros(pad_length, dtype=torch.long, device=device)]))

    return torch.stack(padded_input_ids), torch.stack(padded_attention_mask), torch.tensor(targets)

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained(hp['base_model'])
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
from transformers import GPT2ForSequenceClassification

class ClassifierLayer(torch.nn.Module):

  def __init__(self, input_size, output_size, bias=False):
    super(ClassifierLayer, self).__init__()

    self.dropout = torch.nn.Dropout(0.1)
    self.linear = torch.nn.Linear(input_size, output_size, bias=bias)

  def forward(self, x):
    inputs = self.dropout(x)
    return self.linear(inputs)

class GPT2Classification(GPT2ForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.score = ClassifierLayer(config.n_embd, self.num_labels, bias=False)

        self.post_init()

In [ ]:
from accelerate import Accelerator

# use fp16 mixed precision to improve training speed
accelerator = Accelerator(mixed_precision=hp['mixed_precision']) # fp16
device = accelerator.device

model = GPT2Classification.from_pretrained(hp['base_model'], num_labels=num_labels)
#model = GPT2ForSequenceClassification.from_pretrained(hp['base_model'], num_labels=num_labels)
model.to(device)
tokenizer.pad_token_id = tokenizer.eos_token_id
# fix model padding token id
model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2Classification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# train loop

def train_loop(model, train_loader, val_loader, loss_fct, optimizer, lr_scheduler, progress_bar, log_file_handler, logging_step=1, use_amp=False):
    samples = 0.
    cumulative_loss = 0.

    # set model to train mode
    model.train()

    for step, (inputs, attention_masks, targets) in enumerate(train_loader):
        targets = targets.reshape(-1, 1).to(device)
        attention_masks = attention_masks.to(device)
        outputs = model(inputs, attention_mask=attention_masks)
        loss = loss_fct(outputs["logits"].view(-1, model.num_labels), targets.view(-1))
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

        samples += inputs.shape[0]
        cumulative_loss += loss.item()

        if step % logging_step == 0:
            # calculate qwk on test set

            with torch.no_grad():
                test_loss, test_preds, qwk = test_loop(model, val_loader, loss_fct, use_amp=use_amp)
            model.train()
            log_str = "Step: {:<6} \t Train loss: {:<6.4f} \t Validation loss: {:<6.4f} \t QWK: {:<6.4f}".format(step, (cumulative_loss/samples), test_loss, qwk)
            print(log_str)
            log_file_handler.write(log_str + "\n")
            samples = 0
            cumulative_loss = 0

    return cumulative_loss/samples if samples != 0 else float("inf")

def test_loop(model, test_loader, loss_fct, use_amp=False, show_progression=False):
    samples = 0.
    cumulative_loss = 0.
    preds = []
    labels = []

    # set model to eval mode
    model.eval()

    loop_iterator = enumerate(tqdm(test_loader)) if show_progression else enumerate(test_loader)

    with torch.no_grad():
        for step, (inputs, attention_masks, targets) in loop_iterator:
            targets = targets.reshape(-1, 1).to(device)
            inputs = inputs.to(device)
            attention_masks = attention_masks.to(device)
            with torch.autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(inputs, attention_mask=attention_masks)
            loss = loss_fct(outputs["logits"].view(-1, model.num_labels), targets.view(-1))

            samples += inputs.shape[0]
            cumulative_loss += loss.item()

            probs = outputs['logits'].softmax(-1) # probs
            predictions = probs.argmax(-1) # predicted classes

            labels.extend(targets.tolist())
            preds.extend(predictions.tolist())

        qwk = kappa(preds, labels, weights='quadratic')

    return cumulative_loss/samples if samples != 0 else float("inf"), np.asarray(preds, dtype=np.float32), qwk

In [ ]:
from tqdm.auto  import tqdm
from transformers import get_scheduler
from torch.optim import AdamW
from sklearn.utils import class_weight

lr = hp['lr']
num_epochs = hp['num_epochs']
batch_size = hp['batch_size']
use_amp = hp['use_amp']

# create train data loader
train_dataset = EssayDataset(X_train, y_train, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
num_training_steps = num_epochs * len(train_loader)

# create test data loader
test_dataset = EssayDataset(X_test, y_test, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# create val data loader
val_dataset = EssayDataset(X_val, y_val, tokenizer)
val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# # get class weights
# class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
# class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

loss_fct = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=lr)
lr_scheduler = get_scheduler(
    name="polynomial", optimizer=optimizer,
    num_warmup_steps=0, num_training_steps=num_training_steps
)
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

# use accelerator prepare

# removed val_loader from prepare command
model, optimizer, train_loader, test_loader, val_loader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_loader, test_loader, val_loader, lr_scheduler
)

In [ ]:
def open_log_file(log_folder, essay_df, model, label_column: str, hyper_parameters):
    # using time as a file name for logging

    timestr = time.strftime("%Y%m%d-%H%M%S")
    file_name = timestr + '.log'

    # check if folder exists, create if it isn't
    pathlib.Path(log_folder).mkdir(exist_ok=True, parents=True)

    # open file to log results
    log_file = os.path.join(log_folder, file_name)
    fp = open(log_file, "a")

    fp.write("Log time: " + timestr + "\n")
    fp.write("Essay classes: " + str(essay_df['essay_set'].unique()) + "\n")
    fp.write("Using score column: " + label_column + "\n")
    fp.write("Score distribution: " + "\n" + essay_df[label_column].value_counts().to_string() + "\n")

    fp.write("\n--- Model parameters:\n")
    fp.write(str(model))
    fp.write('\n')

    fp.write("\n--- Hyper parameters:\n")
    for k, v in hyper_parameters.items():
        fp.write(f" {k:<25}: {v}\n")

    fp.write('\n')
    fp.flush()
    return fp

In [ ]:
!nvidia-smi


Tue Oct 10 21:22:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W /  70W |   1213MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
print(torch.cuda.is_available())


True


In [ ]:
                                                                                                                                        # Start logging to a file
fp = open_log_file(log_folder, essay_df, model, label_column=target_column, hyper_parameters=hp)

try:
    # Start training
    progress_bar = tqdm(range(num_training_steps))

    model.train()
    with accelerator.autocast():
        fp.write("Training logs: \n\n")
        for epoch in range(num_epochs):
            train_loss = train_loop(model, train_loader, val_loader, loss_fct, optimizer, lr_scheduler, progress_bar, fp, logging_step=len(train_loader)//3, use_amp=hp['use_amp'])
            with torch.no_grad():
                test_loss, test_preds, qwk = test_loop(model, test_loader, loss_fct)
                log_string = "Epoch: {:<6}\t Test  loss: {:<6.4f} \t QWK: {:<6.4f}".format(epoch+1, test_loss, qwk)
                print(log_string)
                fp.write(log_string + "\n")
finally:
    print("Log file closed.")
    fp.close()

  0%|          | 0/10980 [00:00<?, ?it/s]

Step: 0      	 Train loss: 4.4398 	 Validation loss: 5.1833 	 QWK: -0.0399
Step: 122    	 Train loss: 2.6370 	 Validation loss: 1.4189 	 QWK: 0.0631
Step: 244    	 Train loss: 1.1034 	 Validation loss: 1.0960 	 QWK: 0.0466
Epoch: 1     	 Test  loss: 1.0455 	 QWK: 0.0485
Step: 0      	 Train loss: 0.7189 	 Validation loss: 1.0446 	 QWK: 0.0485
Step: 122    	 Train loss: 0.7990 	 Validation loss: 0.9565 	 QWK: 0.0841
Step: 244    	 Train loss: 0.8252 	 Validation loss: 0.8633 	 QWK: 0.1370
Epoch: 2     	 Test  loss: 0.8834 	 QWK: 0.0736
Step: 0      	 Train loss: 0.4460 	 Validation loss: 0.8827 	 QWK: 0.0736
Step: 122    	 Train loss: 0.7528 	 Validation loss: 0.8408 	 QWK: 0.1392
Step: 244    	 Train loss: 0.7438 	 Validation loss: 0.8067 	 QWK: 0.1935
Epoch: 3     	 Test  loss: 0.7822 	 QWK: 0.2421
Step: 0      	 Train loss: 0.6124 	 Validation loss: 0.7797 	 QWK: 0.2459
Step: 122    	 Train loss: 0.6934 	 Validation loss: 0.7754 	 QWK: 0.2122
Step: 244    	 Train loss: 0.7066 	 Valid